<a href="https://colab.research.google.com/github/fukagai-takuya/gifu-ai/blob/main/gifu-ai-2025-12-21/StableDiffusion3_5_GitClone_Medium_keep_Inference_Instance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Stability AI の Stable Diffusion 3.5 の学習済みモデルを動かす参照実装のコードを Google Drive に git clone

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!mkdir -p /content/drive/MyDrive/sd3.5_custom
%cd /content/drive/MyDrive/sd3.5_custom
!git clone https://github.com/Stability-AI/sd3.5.git

2. Hugging Face から参照実装のコードを動かす学習済みモデルを取得する準備 (どのようなファイル名で学習済みモデルが保存されているかを確認)

In [ ]:
from huggingface_hub import list_repo_files

repo_id = "stabilityai/stable-diffusion-3.5-medium"
files = list_repo_files(repo_id)

for f in files:
    print(f)

3. Hugging Face から参照実装のコードを動かす学習済みモデルを取得


In [ ]:
!mkdir -p /content/models

import os
from getpass import getpass
from huggingface_hub import hf_hub_download

# Stablde Diffusiton 3.5 Medium の学習済みモデルの取得
hf_hub_download(
    repo_id="stabilityai/stable-diffusion-3.5-medium",
    filename="sd3.5_medium.safetensors",
    local_dir="/content/models"
)

# Stablde Diffusiton 3.5 で使用する CLIP-L, CLIP bigG, T5-XXL の学習済みモデルの取得
hf_hub_download(repo_id="stabilityai/stable-diffusion-3.5-medium",
                filename="text_encoders/clip_l.safetensors", local_dir="/content/models")
hf_hub_download(repo_id="stabilityai/stable-diffusion-3.5-medium",
                filename="text_encoders/clip_g.safetensors", local_dir="/content/models")
hf_hub_download(repo_id="stabilityai/stable-diffusion-3.5-medium",
                filename="text_encoders/t5xxl_fp16.safetensors", local_dir="/content/models")

!mv /content/models/text_encoders/clip_g.safetensors /content/models/clip_g.safetensors
!mv /content/models/text_encoders/clip_l.safetensors /content/models/clip_l.safetensors
!mv /content/models/text_encoders/t5xxl_fp16.safetensors /content/models/t5xxl.safetensors


4. 必要な python package をインストール

In [ ]:
%cd /content/drive/MyDrive/sd3.5_custom/sd3.5/
!python3 -s -m pip install -r requirements.txt

5. Stable Diffusion 3.5 Medium を使用した SD3Inferencer のインスタンスを生成して画像を生成 (SD3Inferencer のインスタンス及び画像生成メソッドのパラメータを他のコードセルで参照できるようにグローバル変数保存するよう書き換えた sd3_infer.py を実行します)

In [ ]:
%cd /content/drive/MyDrive/sd3.5_custom/sd3.5/
%run sd3_infer.py --prompt "Stylish modern typography. The phrase 'Gifu AI Study Group' centered on a simple light background." --model /content/models/sd3.5_medium.safetensors --model_folder /content/models --steps 28


6. sd3_infer.py 実行後のグローバル変数を確認

In [ ]:
for k, v in globals().items():
    if not k.startswith('_'):  # _ で始まる内部変数を除く
        print(k, "->", type(v), " - ", v)


7. 生成済みの SD3Inference のインスタンスとグローバル変数に格納したパラメータを使用して画像を生成 (ロード処理の繰り返し実行を避けるため、上記 5. の処理のうち画像生成部分のみを実行するようにしました)

In [ ]:
out_dir = os.path.join("outputs",
                       "sd3.5_medium",
                       os.path.splitext(os.path.basename(SD3_5_sanitized_prompt))[0][:50]
                       + datetime.datetime.now().strftime("_%Y-%m-%dT%H-%M-%S"),
                       )

os.makedirs(out_dir, exist_ok=False)

SD3_5_inferencer.gen_image(
    SD3_5_prompts,
    SD3_5_width,
    SD3_5_height,
    SD3_5_steps,
    SD3_5_cfg,
    SD3_5_sampler,
    SD3_5_seed,
    SD3_5_seed_type,
    out_dir,
    SD3_5_controlnet_cond_image,
    SD3_5_init_image,
    SD3_5_denoise,
    SD3_5_skip_layer_config
)

